In [1]:
import json
import datetime
import collections
from collections import defaultdict

In [ ]:
def threeYears(game_date):
    """Return True if game is within the last three years and Fale is the game is not"""
    three_years_1 = datetime.datetime.today() - datetime.timedelta(days=3*365)
    three_years_2 = three_years_1.isoformat().split("T")[0]
    if three_years_2 < game_date:
        return True
    else:
        return False

In [ ]:
def rival_dict():
    """Return dictionary of team:rivalteams"""
    sqlc = HiveContext(sc)
    individual_game = sc.textFile("s3://ds-fulla/mongo/20150615/statistics/stats/individualgame").map(json.loads)
    # NO NONES 478832
    no_nones = individual_game.filter(lambda x: x[u't']!=None and x[u'g']!=None)\
        .filter(lambda x: x[u'g'][u'o'] and x[u'g'][u'd']!=None and x[u'g']['ws']!=None and x[u'g']['ps']!=None)
    #TEAM OPPONENT SCORE DATE SCORE 478832 FILTER 477501 
    user_game = no_nones.map(lambda x: (int(x[u'u']),int(x[u't']),int(x[u'g'][u'o']),str(x[u'g'][u'd'][u'$date']),x[u'g']['ws']-x[u'g']['ps']))\
        .map(lambda x: ((x[1] if x[1] < x[2] else x[2],x[1] if x[1] > x[2] else x[2]),x[0],x[3].split("T")[0],x[4] if x[1] < x[2] else x[4]*-1))\
        .map(lambda x: (x[0], x[1], x[2], (1 if x[3] >= 0 else 0, 1 if x[3] < 0 else 0)))\
        .filter(lambda x: threeYears(x[2]))
    #TEAM OPPONENT DATE SCORE DISTINCT 45150
    game_count = user_game.map(lambda x: (x[0],x[2],x[3])).distinct()
    #HOW MANY WINS AND LOSSES GREATER THAN ONE EACH MORE THAN 3 GAMES 743
    win_count = game_count.map(lambda x: (x[0],x[2])).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
        .filter(lambda x: x[1][0] > 0 and x[1][1] > 0 and (x[1][0] + x[1][1]) > 2)
    rival_dict = dict(win_count.collect())
    return rival_dict

In [ ]:
def team_dict():
    """Return dictionary of user:teams"""
    athlete_rdd = sc.textFile("s3://ds-fulla/mongo/20150507/monolith/athletes/athletes").map(json.loads)
    team_dict = dict(athlete_rdd.filter(lambda x: len(x['t']) > 0)\
            .map(lambda x: (int(x['_id']), [int(y['tid']) for y in x['t']]))\
            .collect())
    return team_dict

In [ ]:
def user_teams(user, team_dict):
    """Return the teams a user is on"""
    if user not in team_dict.keys():
            return "User not found"
    else:
        teams = team_dict[user]
        return teams

In [ ]:
def user_rivals(user1, user2, rival_dict, team_dict):
    """Return yes if user1 and user2 are rivals and no if they are not"""
    user_team = []
    team1 = user_teams(user1, team_dict)
    team2 = user_teams(user2, team_dict)
    if team1 == "User not found" or team2 =="User not found":
            return "No rivals beause no team"
    else:
        for t1 in team1:
            for t2 in team2:
                user_team.append(tuple(sorted((t1, t2))))
        for ut in user_team:
            if ut in rival_dict.keys():
                return "Rival"
            else:
                return "No rivals because not in dictionary"
        

In [ ]:
rival_dict = rival_dict()
team_dict = team_dict()

In [ ]:
user_rivals(151888,3681735,rival_dict,team_dict)

In [ ]:
user_rivals(151888,543545,rival_dict,team_dict)